# python 简洁之美

## 一行代码交换a,b

In [1]:
a, b = 1, 4

a, b = b, a
a,b

(4, 1)

## 一行代码反转列表

In [2]:
[1,2,3][::-1] 

[3, 2, 1]

## 一行代码合并两个字典

In [3]:
{**{'a':1,'b':2}, **{'c':3}}

{'a': 1, 'b': 2, 'c': 3}

## 一行代码列表去重

In [4]:
set([1,2,2,3,3,3])

{1, 2, 3}

## 一行代码求多个列表中的最大值

In [5]:
max(max([ [1,2,3], [5,1], [4] ], key=lambda v: max(v))) 

5

## 一行代码生成逆序序列

In [6]:
list(range(10,-1,-1))

[10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]

# python概览

## 任务

假设由于每种原因你现在需要去记录人们的信息。你想写个程序来这些人的详细信息。也就是说你想把这些记录存在数据库中，以便在电脑中永久存储这些信息。  

你可以找到现成的数据库管理软件来管理对应的数据库。但是你可以亲自编写这样一个程序，可以完全控制它的运行方式，于是就这样入python的坑.........

## 第一步 表示记录

如果我们想在数据库中存储记录，就要确定以什么样的数据类型来存储。通常使用列表和字典这样的内建对象类型就够用了。

### 试用 List

#### 新建记录

In [7]:
bob = ["Bob Smith", 42, 20000, "software"]
sue = ["Sue James", 30, 30000, "hardware"]

#### 通过对列表的访问，处理可以容易地处理记录

In [8]:
#　Bob　姓啥？
bob[0].split()[-1]

'Smith'

In [9]:
# 给 sue 涨薪 25%
sue[2] *= 1.25
sue

['Sue James', 30, 37500.0, 'hardware']

#### 数据库列表

为了把 Bob 和 Sue 存储在一起，我们可以简单地把他们放入一个新的列表里，形成一个简单的数据库

In [10]:
people = [bob, sue]
# 查看一个这个数据库的内容
for person in people:
    print(person)

['Bob Smith', 42, 20000, 'software']
['Sue James', 30, 37500.0, 'hardware']


通过操作数据库来处理记录

In [11]:
# 给每人涨薪 20%
for person in people:
    print(person[0].split()[-1])  # 打印姓氏
    person[2] *= 1.20   # 涨薪水

Smith
James


In [12]:
# 检查一下有没有涨薪水
for person in people:
    print(person[2])

24000.0
45000.0


#### 使用python迭代工具从记录中获取各种值。例如获取列表解析、map和生成器表达式

In [13]:
# 获取员工的薪水列表
# 列表解析
pays = [person[2] for person in people]
pays

[24000.0, 45000.0]

In [14]:
# map
pays = map((lambda x: x[2]), people)
list(pays)

[24000.0, 45000.0]

In [15]:
# # 查看公司应付员工薪水总额
# 生成器表达式， sum为内建函数
sum(person[2] for person in people)

69000.0

#### Field标签

通过对列表的位置来访问字段导致我们一直需要记住列表每个位置的含义，这样编码麻烦而且代码可读性也差，最好把字段名和字段的值关联起来，也就是咱们给列表的每个位置起个名字。

In [16]:
NAME, AGE, PAY, JOB = range(4)
kobe = ["Kobe Bryant", 60, 60000, "spokesperson"]

In [17]:
# 查看姓名
kobe[NAME]

'Kobe Bryant'

In [18]:
# 查看职位
JOB, kobe[JOB]  

(3, 'spokesperson')

这样兄弟萌就解决了可读性问题，但是我们发现数据记录和字段名并非是关联在一起的，这样随着时间的前进，后来的编码人员对数据结构进行变化时，如果忘了对字段变量做相应的维护的话，它们将会失去同步。  

而且字段名是独立的变量，没有从记录到其字段名的直接映射。  

我们可以尝试利用元组的列表来解决这个问题，其中每个元组保存字段名和值，更加理想的是，使用列表的列表允许进行更新的，（元组是不可变的）

In [19]:
bob = [['name', "Bob Smith"], ["age", 42], ["pay", 30000]]
sue = [['name', "Sue James"], ["age", 46], ["pay", 40000]]
people = [bob, sue]

但是这样我们依然需要通过列表的index值来获取数据。

In [20]:
# 收集人员姓名
[person[0][1] for person in people]

['Bob Smith', 'Sue James']

实际上仅仅是增加了一层额外的位置索引。更好的办法是，可以在循环中检查字段名字来找到我们想要的属性

In [21]:
for person in people:
    for (name, value) in person:
        if name == "name" :
            print(value)

Bob Smith
Sue James


我们可以抽取其中的逻辑写个函数接口来做项工作，以后调用函数进行复用就可以了

In [22]:
def field(record, label):
    """
    record: 记录变量名
    label： 字段名
    """
    for(fname, fvalue) in record:
        if fname == label:  # 根据名字来寻找字段名
            return fvalue

In [23]:
# 获取姓名
field(bob, "name")

'Bob Smith'

In [24]:
# 打印所有年龄
for rec in people:
    print(field(rec, "age"))   #简化了代码，提升了可读性

42
46


如果继续探索下去，最终会实现一套将字段名映射到字段值的完善的记录接口函数。

#### 持久存储问题

列表的确可以用作我们的人员数据库，而且的确它也满足一些小程序的需要。但是他是在内存中，一旦python的进程退出，这些数据就会从内存中消失。

### 试用字典

#### 新建字典数据表示

虽然列表的记录表示方式能够使用，但是我们知道内建的字典对象在关联属性的名字和值方面会更高效和方便

In [25]:
bob = {"name":"Bob Smith", "age":42, "pay":30000, "job":"dev"}
sue = {"name":"sue James", "age":46, "pay":40000, "job":"mng"}

In [26]:
# 相比list我们不再需要记住index值偏移量对应的含义
bob["name"]

'Bob Smith'

In [27]:
# 涨薪  
print(sue["pay"])
sue["pay"] *= 1.10
print(sue["pay"])

# 代码可读性也大大地好
# 字段名和值之间的映射关系也稳定，也不用担心它们之间会失联

40000
44000.0


#### 其他建立字典的方法

可以使用关键词参数和类型构造函数，主要所有的键是字符串

In [28]:
bob = dict(name="Bob Smith", age=42, pay=30000, job="dev")
sue = dict(name="Sue Jones", age=45, pay=40000, job="hdw")
bob

{'name': 'Bob Smith', 'age': 42, 'pay': 30000, 'job': 'dev'}

或者一次一个字段地填写（字典的键是伪随机排列的呦）

In [29]:
wade = {}

In [30]:
wade["name"] = "D.Wade"
wade

{'name': 'D.Wade'}

In [31]:
wade["age"] = "40"
wade

{'name': 'D.Wade', 'age': '40'}

或者用zip函数将 名/值 两个列表链在一起

In [32]:
keys = ["name", "age", "pay", "job"]
values = ["L.J", "35", "35000000$", "basketball player"] 

# 1
list(zip(keys, values))

[('name', 'L.J'),
 ('age', '35'),
 ('pay', '35000000$'),
 ('job', 'basketball player')]

In [33]:
# 2
dict(zip(keys, values))

{'name': 'L.J', 'age': '35', 'pay': '35000000$', 'job': 'basketball player'}

甚至可以通过一个键序列和所有键的可选初始值来创造字典（便于初始化空字典）

In [34]:
# 先把结构创建出来，然后可以在慢慢填入
fields = {"name", "age", "job"}
record = dict.fromkeys(fields, "?")
record

{'job': '?', 'age': '?', 'name': '?'}

#### 字典列表数据库

不管怎样实现字典，仍然需要把基于字典的数据记录会聚在一起，形成数据库。我们依然使用列表

In [35]:
bob

{'name': 'Bob Smith', 'age': 42, 'pay': 30000, 'job': 'dev'}

In [36]:
sue

{'name': 'Sue Jones', 'age': 45, 'pay': 40000, 'job': 'hdw'}

In [37]:
people = [bob, sue]
for person in people:
    print(person["name"], person["pay"])

Bob Smith 30000
Sue Jones 40000


对数据库可以使用迭代工具

In [38]:
# 人员有哪些
names = [person["name"] for person in people]
names

['Bob Smith', 'Sue Jones']

In [39]:
# map
list(map((lambda x: x["name"]), people))

['Bob Smith', 'Sue Jones']

In [40]:
# 总共应付人员薪水
sum(person["pay"] for person in people)

70000

In [41]:
db = {}                 # 数据库
db["bob"] = bob         # 引用字典的字典
db["sue"] = sue

由于字典是python对象，所以数据库中的记录可以用python语法进行增删改查

In [42]:
for person in people:
    person["pay"] *= 1.10  # 为所有人员加薪

#### 嵌套结构

In [43]:
# 现在抽取姓氏时
for person in people:
    print(person["name"].split()[-1])

Smith
Jones


通过再进一步结构化我们的记录，可以避免上述列子中这样的抽取姓氏的代码

In [44]:
# 将字典、列表、元组嵌套在另一个字典中
bob2 = {"name":{"first":"bob", "last":"Smith"},
        "age":42,
        "job":["software", "dancer"],
        "pay":(30000, 20000),
       }

In [45]:
# 获取全名
bob2["name"]

{'first': 'bob', 'last': 'Smith'}

In [46]:
# 获取bob的姓
bob2["name"]["last"]

'Smith'

In [47]:
# bob 的所有工作
for job in bob2["job"] : print(job) 

software
dancer


In [48]:
# bob 又增加了一个工作
bob2["job"].append("lol player")

In [49]:
bob2["job"]

['software', 'dancer', 'lol player']

#### 字典的字典

我们对基于字典的人员数据库做最后一次改进，可以从字典中多挖掘一些字典的好处。  

我们使用字典表示数据库本身，

In [50]:
bob = dict(name="Bob Smith", age=42, pay=30000, job="dev")
sue = dict(name="Sue Jones", age=45, pay=40000, job="hdw")

In [51]:
db = {} # 建立一个数据库

In [52]:
# 引用字典的字典
db["bob"] = bob 
db["sue"] = sue

In [53]:
db["bob"]["name"]

'Bob Smith'

In [54]:
db["sue"]["pay"] * 1.1  # 涨薪

44000.0

In [55]:
# 这样的结构可以直接访问一条记录，而不必循环中寻找到它。
db #查看数据库

{'bob': {'name': 'Bob Smith', 'age': 42, 'pay': 30000, 'job': 'dev'},
 'sue': {'name': 'Sue Jones', 'age': 45, 'pay': 40000, 'job': 'hdw'}}

In [56]:
# python的pprint可以让输出更易于阅读
import pprint
pprint.pprint(db)

{'bob': {'age': 42, 'job': 'dev', 'name': 'Bob Smith', 'pay': 30000},
 'sue': {'age': 45, 'job': 'hdw', 'name': 'Sue Jones', 'pay': 40000}}


如果仍然需要逐条记录地访问数据库，可以使用for循环和字典的迭代器

In [57]:
# 迭代起
x = [db[key]["name"] for key in db]
y = [rec["name"] for rec in db.values()]
print(x, y)

['Bob Smith', 'Sue Jones'] ['Bob Smith', 'Sue Jones']


#### 数据库增删改查

In [58]:
#　数据库的记录总数
len(db)

2

In [59]:
# 增 - 只要把新的记录赋给一个新的键即可
db["tom"] = dict(name="Tom", age=50, job=None, pay=0)
pprint.pprint(db)

{'bob': {'age': 42, 'job': 'dev', 'name': 'Bob Smith', 'pay': 30000},
 'sue': {'age': 45, 'job': 'hdw', 'name': 'Sue Jones', 'pay': 40000},
 'tom': {'age': 50, 'job': None, 'name': 'Tom', 'pay': 0}}


In [60]:
# 查看数据库有哪些人员记录
list(db.keys())

['bob', 'sue', 'tom']

In [61]:
# 类似 SQL 的条件查询
[rec["name"] for rec in db.values() if rec["age"] >= 45] 

['Sue Jones', 'Tom']

#### 持久存储问题

虽然我们的数据库仍然是内存的临时对象，当python进程死亡时，这些数据也随之消失，但是这种字典的结构与永久性对象的系统结构非常切合--shelve(英文语法上应该是shelf，但是python模块命名和术语用的shelve)

## 第二步  持久化存储记录

## 第二步 ....

## 使用pickle将记录保存更新进一个普通文件

In [62]:
# 保存
import pickle
with open("people_pickle", "wb") as dbfile:  # 使用 3. 
    pickle.dump(db, dbfile)                # db 可以单独的以初始化模块文件来组织

In [63]:
# 读取
with open("people_pickle", "rb") as dbfile:  # 使用 3. 
     db_ = pickle.load(dbfile)                
     pprint.pprint(db_)

{'bob': {'age': 42, 'job': 'dev', 'name': 'Bob Smith', 'pay': 30000},
 'sue': {'age': 45, 'job': 'hdw', 'name': 'Sue Jones', 'pay': 40000},
 'tom': {'age': 50, 'job': None, 'name': 'Tom', 'pay': 0}}


## 当处理非常大的数据时，上述方法会变得很慢，因为更新一条数据就要重新读出和写入整个数据库，我们可以通过存储数据库中的一条记录到一个普通的文件的方式来改进

In [64]:
# 1  将每条记录存到对应的文件中
for (key, record) in [("bob", bob), ("sue", sue)]:
    with open(key+".pkl", "wb") as recfile:
        pickle.dump(record, recfile)

In [65]:
# 2  使用glob模块获取整个数据库。它根据文件拓展名来获取这个目录下所有该拓展名的文件。如果需要载入单个记录，可以打开对应的文件并使用pickle反序列化。为了得到一条记录，我们只需要载入一个记录而不需要整个数据库
import glob

# 查看整个数据库
for filename in glob.glob("*.pkl"):    
    with open(filename, "rb") as recfile:
        record = pickle.load(recfile)
        print(filename, "=>\n",record)

#　查看单个记录
suefile = open("sue.pkl", "rb")
print("sue.pkl['name']", "=>\n", pickle.load(suefile)["name"])
suefile.close()

bob.pkl =>
 {'name': 'Bob Smith', 'age': 42, 'pay': 30000, 'job': 'dev'}
sue.pkl =>
 {'name': 'Sue Jones', 'age': 45, 'pay': 40000, 'job': 'hdw'}
sue.pkl['name'] =>
 Sue Jones


In [66]:
# 3  当更新数据库，从对应文件抓取一条记录，在内存中进行修改，然后重新写入它的pickle文件，不用为了更新某段记录而对整个数据库进行操作

suefile = open("sue.pkl", "rb")
sue = pickle.load(suefile)
suefile.close()

sue["pay"] *= 1.10  # 给sue涨薪10%
suefile = open("sue.pkl", "wb")
pickle.dump(sue, suefile)
suefile.close()

# 查看该记录是否更新成功
suefile = open("sue.pkl", "rb")
print("sue.pkl['pay']", "=>\n", pickle.load(suefile)["pay"])
suefile.close()

sue.pkl['pay'] =>
 44000.0


上述方法数据库的中的key变为了实际的文件名。也就是说文件系统成为了高层字典，通过文件名可以直接访问每条记录

## shelves 
自动地将对象pickle进和pickle出健访问文件系统。它们很像必须打开着的字典，存储持久化对象的字典 

In [67]:
# 创建数据库
import shelve
db = shelve.open("people-shelve") 
db["bob"] = bob
db["sue"] = sue
db.close()

In [68]:
# 访问数据库, 依旧是字典的字典
db = shelve.open("people-shelve")
for key in db:
    print(key, "=>\n", db[key])

bob =>
 {'name': 'Bob Smith', 'age': 42, 'pay': 30000, 'job': 'dev'}
sue =>
 {'name': 'Sue Jones', 'age': 45, 'pay': 44000.0, 'job': 'hdw'}
tom =>
 {'name': 'Tom', 'age': 50, 'job': None, 'pay': 0}


In [69]:
# 更新
tom= dict(name="Tom", age=50, job=None, pay=0)

db = shelve.open("people-shelve")
sue = db["sue"]
sue["pay"] *= 1.1
# 手动写回是缺省需求，可以在open中定义关闭shelve时写回，但是会消耗内存使得关闭变慢
# 缺省是需要时创建，否则以可读可写方式打开 
db["sue"] = sue   
db["tom"] = tom
db.close() # 关闭shelve、关闭缓存

# 可以将数据库操作单独制作成脚本文件，通过运行相应程序 自动对本地数据库操作并将变化状态持久化地保存在文件中，

## OOP

In [70]:
# 实例化类的方式来实现记录数据库
class Person:
    def __init__(self, name, age, pay=0, job=None):
        self.name = name
        self.age = age
        self.pay = pay
        self.job = job

In [71]:
bob = Person("Bob Smith", 42, 30000, "software")
sue = Person("Sue Jones", 45, 35000, "hartware")
bob.name.split()[-1] # 获取姓氏

'Smith'

In [72]:
# 涨薪
sue.pay *= 1.1
sue.pay

38500.0

In [73]:
# 将类的实例放入列表和字典，构成一个整体
bob = Person("Bob Smith", 42, 30000, "software")
sue = Person("Sue Jones", 45, 35000, "hartware")

people = [bob, sue]  # 构成一个数据库列表
for person in people:
    print(person.name, person.pay)

Bob Smith 30000
Sue Jones 35000


In [74]:
x = [(person.name, person.age, person.pay,  person.job) for person in people]
x

[('Bob Smith', 42, 30000, 'software'), ('Sue Jones', 45, 35000, 'hartware')]

In [75]:
# SQL 风格查询
[rec.name for rec in people if rec.age >= 45]

['Sue Jones']

In [76]:
#  为类添加行为
class Person:
    def __init__(self, name, age, pay=0, job=None):
        self.name = name
        self.age = age
        self.pay = pay
        self.job = job
    
    def get_LastName(self):
        return self.name.split()[-1]

    def giveRaise(self, percent=0.1):
        self.pay *= (1.0 + percent)

In [77]:
bob = Person("Bob Smith", 42, 30000, "software")
sue = Person("Sue Jones", 45, 35000, "hartware")

bob.get_LastName()

'Smith'

In [78]:
print(sue.pay)
sue.giveRaise()
print(sue.pay)

35000
38500.0


In [79]:
# 继承  通过人员类型编写子类
class Manager(Person):
    
    # 比如管理层除了正常涨薪还有 额外百分比的奖金的话
    # 重写该功能
    def giveRaise(self, percent=0.1, bonus=0.1) :
        self.pay *= (1.0 + percent +bonus)

In [80]:
tom = Manager(name="Tom Doe", age=50, pay=50000)
tom.get_LastName() # 父类的功能

'Doe'

In [81]:
tom.giveRaise(bonus=.20) # 自定义的涨薪功能
tom.pay

65000.0

## 重构

In [82]:
# 增强方法
class Manager(Person):
    
    def giveRaise(self, percent=0.1, bonus=0.1) :
        Person.giveRaise(self, percent+bonus)
# 1.调用父类的方法显式地传入self参数
# 2. 虽然重新定义了该方法，但我们调用的是通用的方法，不但可以降低冗余（业务逻辑只出现一次）
# 3.实践中易于启动父类的构造器

In [83]:
# 格式化显示
class Person:
    def __init__(self, name, age, pay=0, job=None):
        self.name = name
        self.age = age
        self.pay = pay
        self.job = job
    # 操作符重载
    def __str__(self):
        # 自定义返回值,对当对象需要作为整体打印时，比实例缺省的显示好得多
        return "<%s => %s: %s, %s>" % (self.__class__.__name__, self.name, self.job, self.pay)

    def get_LastName(self):
        return self.name.split()[-1]
    
    # 可以编写 __add__ 使得 + 表达式自动调用 giveraise方法，代价是代码可读性变差，不知到具体你 + 了啥？
    def giveRaise(self, percent=0.1):
        self.pay *= (1.0 + percent)


class Manager(Person):
    
    def __init__(self, name, age, pay):
        Person.__init__(self, name, age, pay=0, job="Manager")

    def giveRaise(self, percent=0.1, bonus=0.1) :
        Person.giveRaise(self, percent+bonus)

In [84]:
bob = Person("Bob Smith", 44)
sue = Person("Sue Jones", 47, 40000, "Hardware")
tom = Manager(name="Tom Doe", age=50, pay=50000)
for obj in (bob, sue, tom):
    obj.giveRaise(.10)
    print(obj)

<Person => Bob Smith: None, 0.0>
<Person => Sue Jones: Hardware, 44000.0>
<Manager => Tom Doe: Manager, 0.0>


In [85]:
# 增加持久化
bob = Person("Bob Smith", 44)
sue = Person("Sue Jones", 47, 40000, "Hardware")
tom = Manager(name="Tom Doe", age=50, pay=50000)

# 创建了类实例的shelve、然后赋给新建的shelve文件的键
db = shelve.open("class-shelve")
db["bob"] = bob
db["sue"] = sue
db["tom"] = tom
db.close()

In [86]:
# 检验数据库
db = shelve.open("class-shelve")
for key in db:
    print(key, " =>\n ", db[key].name, db[key].pay)

bob  =>
  Bob Smith 0
sue  =>
  Sue Jones 40000
tom  =>
  Tom Doe 0


In [87]:
bob = db["bob"]
print(bob.get_LastName())  # 不需要导入类相关类，会自动定位相关类

Smith
